In [1]:
# !pip install -q langchain langchain_community sentence_transformers langchain-huggingface faiss-cpu


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
loader = WebBaseLoader("https://en.wikipedia.org/wiki/Main_Page")
documents = loader.load()

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

D:\WorksOfGreatness\rag-cvpr-ask\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=4000)
llm = HuggingFacePipeline(pipeline=generation_pipeline)

In [3]:
from langchain.prompts import PromptTemplate

# Define a custom prompt template for the RAG model
prompt_template = """Given the following context, answer the question.

Context: {context}

Question: {question}

Answer:"""

prompt = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
rag_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff",
                                      retriever=vectorstore.as_retriever(),
                                      chain_type_kwargs={"prompt": prompt})


NameError: name 'llm' is not defined

In [16]:
question = "What is the paper about"
result = rag_chain({"query": question})
result["result"]

TypeError: Chain.__call__() got an unexpected keyword argument 'max_new_tokens'